In [19]:
import pandas as pd
import io, requests, os 
from bs4 import BeautifulSoup
from tqdm import tqdm
pd.set_option('display.max_columns', None)

start, end = "2020-02-29", "2022-12-30"
output_folder = 'CovidDec30-2022'
if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok=True)

def valid_date(date):
    try:
        pd.to_datetime(date)
        return True
    except:
        return False

# PVI data

In [8]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/COVID19PVI/data/master/Model11.2.1/2020/Model_11.2.1_20200228_data.csv", 
    header=12
)
df.head(2)

,row,sid,casrn,name,Spread,Sick,PctCases,Cases,CountyCaseRate,StateCaseRate,PctDeaths,Deaths,PctDeaths2,CountyDeathRate,StateDeathRate,StatePctTested,StateTestRate,SVISocioeconomic,SVIMinority,SVIHousing,PctBeds,Mobility,PctNoIns,PctGE65,DaytimePopDensity,PrematureDeath,Smoking,AirPollution,Diabetes,Traffic,DistancingGrade,Obesity,PctBlack,PctNative
0,1,"-86.6441,32.5395",1001,"Alabama, Autauga",NaN,0.0,0.0,0,NaN,NaN,NaN,0,0.0,NaN,NaN,0.000023,1.242837,0.3631,0.5947,0.3741,0.001529,1733.08,7.1,14.6,76.0,8128.591190,0.180816,11.7,0.111,88.457040,5.0,0.333,0.193432,0.004802
1,2,"-87.7221,30.7277",1003,"Alabama, Baldwin",NaN,0.0,0.0,0,NaN,NaN,NaN,0,0.0,NaN,NaN,0.000023,1.242837,0.2232,0.4358,0.3359,0.001523,1971.57,8.9,19.5,121.0,7354.122530,0.174890,10.3,0.107,86.997430,5.0,0.310,0.087840,0.007724
2,3,"-85.3871,31.8683",1005,"Alabama, Barbour",NaN,0.0,0.0,0,NaN,NaN,NaN,0,0.0,NaN,NaN,0.000023,1.242837,0.9780,0.8558,0.9889,0.002974,1080.54,11.3,18.0,27.5,10253.573403,0.220000,11.5,0.176,102.291762,4.3,0.417,0.480326,0.006591
3,4,"-87.1251,32.9964",1007,"Alabama, Bibb",NaN,0.0,0.0,0,NaN,NaN,NaN,0,0.0,NaN,NaN,0.000023,1.242837,0.7694,0.4323,0.7189,0.001563,1272.47,10.7,16.3,28.9,11977.539484,0.191142,11.2,0.145,29.335580,5.0,0.376,0.211205,0.004375
4,5,"-86.5679,33.9821",1009,"Alabama, Blount",NaN,0.0,0.0,0,NaN,NaN,NaN,0,0.0,NaN,NaN,0.000023,1.242837,0.6143,0.5915,0.1741,0.000692,1395.46,10.8,17.8,71.7,11335.071134,0.192087,11.7,0.170,33.411782,5.0,0.338,0.014627,0.006535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,3138,"-108.8828,41.6594",56037,"Wyoming, Sweetwater",NaN,0.0,0.0,0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,0.2576,0.6718,0.4120,0.002671,2052.42,11.3,10.7,4.5,7831.827655,0.180734,5.1,0.088,154.754714,3.7,0.298,0.011452,0.015447
3138,3139,"-110.5891,43.9352",56039,"Wyoming, Teton",NaN,0.0,0.0,0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,0.0025,0.7736,0.6266,0.002080,3165.12,12.7,13.6,6.7,2730.603992,0.145464,4.9,0.022,135.188600,4.0,0.124,0.006326,0.009142
3139,3140,"-110.5476,41.2878",56041,"Wyoming, Uinta",NaN,0.0,0.0,0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,0.3583,0.4390,0.6657,0.002069,1920.16,11.2,12.1,9.8,7330.683549,0.172127,5.9,0.113,96.191604,3.7,0.358,0.006848,0.014188
3140,3141,"-107.6802,43.9045",56043,"Wyoming, Washakie",NaN,0.0,0.0,0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,0.4293,0.3687,0.2751,0.002283,1723.80,15.0,20.7,3.8,6586.049959,0.168594,4.8,0.117,82.822155,3.7,0.288,0.004692,0.018136


In [9]:
# read in fips codes
fips = pd.read_csv(
    "https://raw.githubusercontent.com/COVID19PVI/data/master/Model11.2.1/2020/Model_11.2.1_20200228_data.csv", 
    header=12, usecols=["name","casrn"]
).rename(columns={'name':'Name','casrn':'FIPS'})


In [10]:
# get the PVI data links
r = requests.get("https://github.com/COVID19PVI/data/tree/master/Model11.2.1")
soup = BeautifulSoup(r.content, 'html.parser')
links = [
    x.attrs["href"].replace("blob","raw")
    for x in soup.find_all('a', {'class':'js-navigation-open Link--primary'})[1:-1]
]

# only download the result files
links = [link for link in links if link.endswith('_results.csv')]

In [ ]:
def filter_links(
    links, start="2020-02-29", end="2022-12-30"
):
    dates = []
    selected_links = []

    for link in links:
        i = link.split("_")[-2]
        date = f"{i[:4]}-{i[4:6]}-{i[6:]}"
        dates.append(date)
        if start <= date <= end:
            selected_links.append(link)

    print(f"First date {min(dates)}, last date {max(dates)}.")
    # print(f"Returning results within {start} and {end}.")
    return selected_links

In [23]:
all_links = []
print(f"Will filter results outside {start} and {end}.")

for folder in ['', '/2020', '/2021', '/2022']:
    r = requests.get(f"https://github.com/COVID19PVI/data/tree/master/Model11.2.1{folder}")
    soup = BeautifulSoup(r.content, 'html.parser')
    links = [
        x.attrs["href"].replace("blob","raw")
        for x in soup.find_all('a', {'class':'js-navigation-open Link--primary'})[1:-1]
    ]

    # only download the result files
    links = [link for link in links if link.endswith('_results.csv')]
    links = filter_links(links)
    all_links.extend(links)

Will filter results outside 2020-02-29 and 2022-12-30.
First date 2022-12-01, last date 2023-01-10.
First date 2020-02-28, last date 2020-12-30.
First date 2021-01-01, last date 2021-12-30.
First date 2022-01-01, last date 2022-11-29.


In [40]:
# combine data
all_data = []
read_from_local = True # Set to true if github isn't responding

if read_from_local:
    # PVI git repo https://github.com/COVID19PVI/data cloned here in local machine
    root_dir = '../../data'
    
    for link in tqdm(all_links):
        # link : '/COVID19PVI/data/raw/master/Model11.2.1/2022/Model_11.2.1_20221129_results.csv'
        # filepath: 'Model11.2.1/2022/Model_11.2.1_20221129_results.csv'
        filepath = link.split('raw/master')[-1]

        data = pd.read_csv(f'{root_dir}/{filepath}')
        data["date"] = filepath.split("/")[-1].split('_')[2]
        all_data.append(data)
else:
    for link in tqdm(all_links):
        data = pd.read_csv("https://github.com"+link)
        data["date"] = link.split("_")[2]
        all_data.append(data)

all_data = pd.concat(all_data)
all_data["date"] = [f"{i[:4]}-{i[4:6]}-{i[6:]}" for i in all_data.date]
all_data.head()

100%|██████████| 1033/1033 [00:13<00:00, 76.32it/s]


,ToxPi Score,HClust Group,KMeans Group,Name,Source,Infection Rate: Transmissible Cases!25!0xcc3333ff,Infection Rate: Disease Spread!5!0xe64d4dff,Pop Concentration: Pop Mobility!10!0x57b757ff,Pop Concentration: Residential Density!10!0x5ced5cff,Intervention: Social Distancing!10!0x4258c9ff,Intervention: Testing!10!0x6079f7ff,Health & Environment: Pop Demographics!10!0x6b0b9eff,Health & Environment: Air Pollution!10!0x8e26c4ff,Health & Environment: Age Distribution!10!0x9a42c8ff,Health & Environment: Co-morbidities!10!0xb460e0ff,Health & Environment: Health Disparities!10!0xc885ecff,Health & Environment: Hospital Beds!5!0xdeb9f1ff,date
0,0.649660,7,10,"Arizona, Apache","-109.4892,35.3947",0.875830,0.456579,0.416114,0.9118,0.675,0.642159,0.695642,0.161677,0.501651,0.563568,0.824094,0.622365,2022-12-01
1,0.640688,5,5,"Virginia, Petersburg City","-77.3937,37.2053",0.617803,0.140093,0.758512,0.8898,1.000,0.697230,0.695818,0.371257,0.544980,0.652553,0.640449,0.286892,2022-12-01
2,0.630193,5,8,"New York, Bronx","-73.8628,40.8521",0.731153,0.258654,0.934726,0.9704,1.000,0.230993,0.771682,0.437126,0.431653,0.301974,0.621230,0.440840,2022-12-01
3,0.626528,10,7,"Mississippi, Pike","-90.4039,31.1751",0.524592,0.078354,0.585140,0.9659,1.000,1.000000,0.642341,0.371257,0.542700,0.529290,0.660993,0.366637,2022-12-01
4,0.626273,7,10,"Arizona, Navajo","-110.3219,35.3998",0.838384,0.409635,0.493231,0.9618,0.575,0.642159,0.692396,0.161677,0.556173,0.514271,0.669091,0.523677,2022-12-01


In [41]:
drop_columns = [
    'Name', 'Source','date', 'ToxPi Score', 'HClust Group', 'KMeans Group', 'Hospital Beds',
    'Pop Mobility', 'Co-morbidities', 'Pop Demographics', 'Air Pollution', 'Residential Density', 'Testing',
    'Age Distribution', 'Health Disparities', 'Transmissible Cases'
]
for column in all_data.columns:
    column_name = column.split('!')[0]
    if ":" in column_name:
        column_name = column_name.split(": ")[1]

    if column_name in drop_columns:
        continue
    print(f"Saving {column_name}", end = "... ")
    mx = all_data.pivot(index="Name",columns="date", values=column).reset_index()
    mx = pd.merge(mx, fips, on="Name", how="left")
    mx.insert(0, "FIPS", mx.pop("FIPS"))
    mx.to_csv(f"{output_folder}/{column_name}.csv", index=False, float_format='%.4f')
    print(u'\u2705')

Saving Disease Spread... ✅
Saving Social Distancing... ✅


In [42]:
all_data.columns

Index(['ToxPi Score', 'HClust Group', 'KMeans Group', 'Name', 'Source',
       'Infection Rate: Transmissible Cases!25!0xcc3333ff',
       'Infection Rate: Disease Spread!5!0xe64d4dff',
       'Pop Concentration: Pop Mobility!10!0x57b757ff',
       'Pop Concentration: Residential Density!10!0x5ced5cff',
       'Intervention: Social Distancing!10!0x4258c9ff',
       'Intervention: Testing!10!0x6079f7ff',
       'Health & Environment: Pop Demographics!10!0x6b0b9eff',
       'Health & Environment: Air Pollution!10!0x8e26c4ff',
       'Health & Environment: Age Distribution!10!0x9a42c8ff',
       'Health & Environment: Co-morbidities!10!0xb460e0ff',
       'Health & Environment: Health Disparities!10!0xc885ecff',
       'Health & Environment: Hospital Beds!5!0xdeb9f1ff', 'date'],
      dtype='object')

# Cases
Source [USAFacts](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/). Data collection starts from 2020-01-22. Note that data from today can take 3 days to update in this file.

In [43]:
response = requests.get("https://static.usafacts.org/public/data/covid-19/covid_confirmed_usafacts.csv")

file_object = io.StringIO(response.content.decode('utf-8'))
df = pd.read_csv(file_object)
df.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,2020-07-06,2020-07-07,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16,2020-07-17,2020-07-18,2020-07-19,2020-07-20,2020-07-21,2020-07-22,2020-07-23,2020-07-24,2020-07-25,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31,2020-09-01,2020-09-02,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,2020-09-29,2020-09-30,2020-10-01,2020-10-02,2020-10-03,2020-10-04,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-12,2020-10-13,2020-10-14,2020-10-15,2020-10-16,2020-10-17,2020-10-18,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01,2020-11-02,2020-11-03,2020-11-04,2020-11-05,2020-11-06,2020-11-07,2020-11-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-15,2020-11-16,2020-11-17,2020-11-18,2020-11-19,2020-11-20,2020-11-21,2020-11-22,2020-11-23,2020-11-24,2020-11-25,2020-11-26,2020-11-27,2020-11-28,2020-11-29,2020-11-30,2020-12-01,2020-12-02,2020-12-03,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-09,2021-01-10,2021-01-11,2021-01-12,2021-01-13,2021-01-14,2021-01-15,

In [44]:
df.drop(columns=['County Name', 'State', 'StateFIPS'], inplace=True)
df.rename({'countyFIPS':'FIPS'}, axis=1, inplace=True)

df = df[(~df['FIPS'].isna()) & (df['FIPS']>0)]
df.fillna(0, inplace=True)
df.to_csv(f'{output_folder}/Cases.csv', index=False)

# Transmissible Cases

Cases from the last 14 days per 100k population. 

In [59]:
cases = pd.read_csv(f'{output_folder}/Cases.csv')
population = pd.read_csv('Support files/Population.csv')

In [60]:
df = population[['FIPS', 'POPESTIMATE']].merge(cases, how='inner', on='FIPS')
date_columns = [col for col in df.columns if valid_date(col)]
print(df.shape)
df.head(3)

(3142, 1079)


,FIPS,POPESTIMATE,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,2020-07-06,2020-07-07,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16,2020-07-17,2020-07-18,2020-07-19,2020-07-20,2020-07-21,2020-07-22,2020-07-23,2020-07-24,2020-07-25,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31,2020-09-01,2020-09-02,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,2020-09-29,2020-09-30,2020-10-01,2020-10-02,2020-10-03,2020-10-04,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-12,2020-10-13,2020-10-14,2020-10-15,2020-10-16,2020-10-17,2020-10-18,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01,2020-11-02,2020-11-03,2020-11-04,2020-11-05,2020-11-06,2020-11-07,2020-11-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-15,2020-11-16,2020-11-17,2020-11-18,2020-11-19,2020-11-20,2020-11-21,2020-11-22,2020-11-23,2020-11-24,2020-11-25,2020-11-26,2020-11-27,2020-11-28,2020-11-29,2020-11-30,2020-12-01,2020-12-02,2020-12-03,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-09,2021-01-10,2021-01-11,2021-01-12,2021-01-13,2021-01-14,2021-01-15,2021-01-16,2021-01-17,

In [61]:
period = 14
forward_shifted = df[date_columns].shift(period, axis=1).fillna(0)

cases_in_last_14_days = df[date_columns[period:]] - forward_shifted[date_columns[period:]]
df[date_columns[period:]] = cases_in_last_14_days.divide(df['POPESTIMATE']/1e5, axis=0)
df[date_columns[:period]] = df[date_columns[:period]].divide(df['POPESTIMATE']/1e5, axis=0
)
df.drop(columns=['POPESTIMATE'], axis=1, inplace=True)
df.head(3)

,FIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,2020-07-06,2020-07-07,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16,2020-07-17,2020-07-18,2020-07-19,2020-07-20,2020-07-21,2020-07-22,2020-07-23,2020-07-24,2020-07-25,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31,2020-09-01,2020-09-02,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,2020-09-29,2020-09-30,2020-10-01,2020-10-02,2020-10-03,2020-10-04,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-12,2020-10-13,2020-10-14,2020-10-15,2020-10-16,2020-10-17,2020-10-18,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01,2020-11-02,2020-11-03,2020-11-04,2020-11-05,2020-11-06,2020-11-07,2020-11-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-15,2020-11-16,2020-11-17,2020-11-18,2020-11-19,2020-11-20,2020-11-21,2020-11-22,2020-11-23,2020-11-24,2020-11-25,2020-11-26,2020-11-27,2020-11-28,2020-11-29,2020-11-30,2020-12-01,2020-12-02,2020-12-03,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-09,2021-01-10,2021-01-11,2021-01-12,2021-01-13,2021-01-14,2021-01-15,2021-01-16,2021-01-17,2021-01-18,2

In [62]:
drop_columns = [
    col for col in date_columns if col < start or col >end
]
df.drop(columns=drop_columns, inplace=True)

df.round(3).to_csv(f'{output_folder}/Transmissible Cases.csv', index=False)

# Disease Spread

Cases that are from the last 14 days (one incubation period) divided by cases from the last 28 days .

In [50]:
df = pd.read_csv(f'{output_folder}/Cases.csv')

period = 14
shifted = df[date_columns].shift(period, axis=1).fillna(0)
cases_difference = df[date_columns[period:]] - shifted[date_columns[period:]]
df_14 = df.copy()
df_14[date_columns[period:]] = cases_difference

period = 28
shifted = df[date_columns].shift(period, axis=1).fillna(0)
cases_difference = df[date_columns[period:]] - shifted[date_columns[period:]]
df_28 = df.copy()
df_28[date_columns[period:]] = cases_difference

In [51]:
df[date_columns] = df_14[date_columns].divide(df_28[date_columns], axis=1).fillna(0)
df.head(3)

,FIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,2020-07-06,2020-07-07,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16,2020-07-17,2020-07-18,2020-07-19,2020-07-20,2020-07-21,2020-07-22,2020-07-23,2020-07-24,2020-07-25,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31,2020-09-01,2020-09-02,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,2020-09-29,2020-09-30,2020-10-01,2020-10-02,2020-10-03,2020-10-04,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-12,2020-10-13,2020-10-14,2020-10-15,2020-10-16,2020-10-17,2020-10-18,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01,2020-11-02,2020-11-03,2020-11-04,2020-11-05,2020-11-06,2020-11-07,2020-11-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-15,2020-11-16,2020-11-17,2020-11-18,2020-11-19,2020-11-20,2020-11-21,2020-11-22,2020-11-23,2020-11-24,2020-11-25,2020-11-26,2020-11-27,2020-11-28,2020-11-29,2020-11-30,2020-12-01,2020-12-02,2020-12-03,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-09,2021-01-10,2021-01-11,2021-01-12,2021-01-13,2021-01-14,2021-01-15,2021-01-16,2021-01-17,2021-01-18,2

In [57]:
drop_columns = [
    col for col in date_columns if col < start or col >end
]
df.drop(columns=drop_columns, inplace=True)

df.round(4).to_csv(f'{output_folder}/Disease Spread.csv', index=False)

# Deaths
Source [USAFacts](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/). Data collection starts from 2020-01-22.

In [4]:
# response = requests.get("https://static.usafacts.org/public/data/covid-19/covid_deaths_usafacts.csv")
# file_object = io.StringIO(response.content.decode('utf-8'))
# df = pd.read_csv(file_object)
# df.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-10-16,2022-10-17,2022-10-18,2022-10-19,2022-10-20,2022-10-21,2022-10-22,2022-10-23,2022-10-24,2022-10-25
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,228,228,228,228,228,228,228,228,228,228
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,715,715,715,715,716,716,716,716,716,716
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,103,103,103,103,103,103,103,103,103,103
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,108,108,108,108,108,108,108,108,108,108


In [5]:
# df.drop(columns=['County Name', 'State', 'StateFIPS'], inplace=True)
# df.rename({'countyFIPS':'FIPS'}, axis=1, inplace=True)

# df = df[(~df['FIPS'].isna()) & (df['FIPS']>0)]
# df.fillna(0, inplace=True)
# df.to_csv(f'{output_folder}/Deaths.csv', index=False)

# Vaccination
From [CDC vaccinations in the US counties](https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh). Data collection starts from 2020-12-13. Note that data from today can take upto 3 days to update in the CDC server.

In [8]:
response = requests.get("https://data.cdc.gov/api/views/8xkx-amqh/rows.csv?accessType=DOWNLOAD")
file_object = io.StringIO(response.content.decode('utf-8'))
df = pd.read_csv("COVID-19_Vaccinations_in_the_United_States_County.csv", low_memory=False)
df.head()

,Date,FIPS,MMWR_week,Recip_County,Recip_State,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,Administered_Dose1_Recip_12Plus,Administered_Dose1_Recip_12PlusPop_Pct,Administered_Dose1_Recip_18Plus,Administered_Dose1_Recip_18PlusPop_Pct,Administered_Dose1_Recip_65Plus,Administered_Dose1_Recip_65PlusPop_Pct,Series_Complete_Yes,Series_Complete_Pop_Pct,Series_Complete_5Plus,Series_Complete_5PlusPop_Pct,Series_Complete_5to17,Series_Complete_5to17Pop_Pct,Series_Complete_12Plus,Series_Complete_12PlusPop_Pct,Series_Complete_18Plus,Series_Complete_18PlusPop_Pct,Series_Complete_65Plus,Series_Complete_65PlusPop_Pct,Booster_Doses,Booster_Doses_Vax_Pct,Booster_Doses_5Plus,Booster_Doses_5Plus_Vax_Pct,Booster_Doses_12Plus,Booster_Doses_12Plus_Vax_Pct,Booster_Doses_18Plus,Booster_Doses_18Plus_Vax_Pct,Booster_Doses_50Plus,Booster_Doses_50Plus_Vax_Pct,Booster_Doses_65Plus,Booster_Doses_65Plus_Vax_Pct,Second_Booster_50Plus,Second_Booster_50Plus_Vax_Pct,Second_Booster_65Plus,Second_Booster_65Plus_Vax_Pct,SVI_CTGY,Series_Complete_Pop_Pct_SVI,Series_Complete_5PlusPop_Pct_SVI,Series_Complete_5to17Pop_Pct_SVI,Series_Complete_12PlusPop_Pct_SVI,Series_Complete_18PlusPop_Pct_SVI,Series_Complete_65PlusPop_Pct_SVI,Metro_status,Series_Complete_Pop_Pct_UR_Equity,Series_Complete_5PlusPop_Pct_UR_Equity,Series_Complete_5to17Pop_Pct_UR_Equity,Series_Complete_12PlusPop_Pct_UR_Equity,Series_Complete_18PlusPop_Pct_UR_Equity,Series_Complete_65PlusPop_Pct_UR_Equity,Booster_Doses_Vax_Pct_SVI,Booster_Doses_12PlusVax_Pct_SVI,Booster_Doses_18PlusVax_Pct_SVI,Booster_Doses_65PlusVax_Pct_SVI,Booster_Doses_Vax_Pct_UR_Equity,Booster_Doses_12PlusVax_Pct_UR_Equity,Booster_Doses_18PlusVax_Pct_UR_Equity,Booster_Doses_65PlusVax_Pct_UR_Equity,Census2019,Census2019_5PlusPop,Census2019_5to17Pop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop,Bivalent_Booster_5Plus,Bivalent_Booster_5Plus_Pop_Pct,Bivalent_Booster_12Plus,Bivalent_Booster_12Plus_Pop_Pct,Bivalent_Booster_18Plus,Bivalent_Booster_18Plus_Pop_Pct,Bivalent_Booster_65Plus,Bivalent_Booster_65Plus_Pop_Pct
0,01/04/2023,18157,1.0,Tippecanoe County,IN,98.6,131034.0,66.9,130134.0,70.6,124944.0,74.0,117602.0,75.7,21860.0,93.9,117257.0,59.9,116993.0,63.5,10795.0,37.3,112715.0,66.8,106198.0,68.3,20568.0,88.4,60408.0,51.5,60406.0,51.6,59497.0,52.8,57193.0,53.9,28912.0,70.0,16351.0,79.5,14825.0,51.3,9855.0,60.3,B,8.0,6.0,6.0,6.0,7.0,7.0,Metro,2.0,2.0,2.0,3.0,3.0,4.0,8.0,8.0,8.0,7.0,4.0,4.0,4.0,3.0,195732.0,184368.0,28962.0,168768.0,155406.0,23280.0,18383.0,10.0,18037.0,10.7,17560.0,11.3,8043.0,34.5
1,01/04/2023,48185,1.0,Grimes County,TX,98.9,17796.0,61.6,17764.0,65.2,17403.0,70.3,16470.0,73.2,4783.0,93.2,15680.0,54.3,15678.0,57.6,1045.0,22.1,15401.0,62.2,14633.0,65.0,4362.0,85.0,6834.0,43.6,6834.0,43.6,6812.0,44.2,6754.0,46.2,5076.0,58.2,2895.0,66.4,1914.0,37.7,1289.0,44.5,D,16.0,14.0,13.0,14.0,14.0,15.0,Non-metro,6.0,6.0,5.0,6.0,7.0,8.0,15.0,15.0,15.0,15.0,7.0,7.0,7.0,7.0,28880.0,27231.0,4733.0,24747.0,22498.0,5134.0,1730.0,6.4,1725.0,7.0,1708.0,7.6,1019.0,19.8
2,01/04/2023,17131,1.0,Mercer County,IL,98.0,10683.0,69.2,10663.0,72.7,10377.0,77.5,9777.0,80.3,3486.0,95.0,9670.0,62.6,9656.0,65.8,811.0,32.5,9393.0,70.1,8845.0,72.7,3220.0,95.0,5141.0,53.2,5141.0,53.2,5111.0,54.4,4975.0,56.2,3918.0,67.1,2519.0,78.2,2137.0,54.5,1618.0,64.2,A,4.0,3.0,2.0,2.0,3.0,3.0,Metro,2.0,3.0,2.0,3.0,3.0,4.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,3.0,15437.0,14668.0,2495.0,13393.0,12173.0,3365.0,2371.0,16.2,2363.0,17.6,2340.0,19.2,1589.0,47.2
3,01/04/2023,08091,1.0,Ouray County,CO,97.0,3826.0,77.3,3784.0,78.7,3647.0,81.1,3473.0,82.8,1320.0,95.0,3442.0,69.5,3409.0,70.9,280.0,45.8,3291.0,73.1,3129.0,74.6,1195.0,88.5,2310.0,67.1,2310.0,67.8,2280.0,69.3,2204.0,70.4,1610.0,78.0,973.0,81.4,1101.0,68.4,724.0,74.4,A,4.0,3.0,3.0,3.0,3.0,3.0,Non-metro,7.0,7.0,7.0,7.0,7.0,8.0,4.0,4.0,4.0,4.0,8.0,8.0,8.0,8.0,4952.0,4808.0,612.0,4499.0,4196.0,1350.0,1218.0,25.3,1199.0,26.7,1165.

In [11]:
df['FIPS'].nunique()

3224

In [12]:
df = df[(df['FIPS']!='UNK') & (~df['FIPS'].isna())]
# df.dropna(axis=0, inplace=True)
df = df[['Date', 'FIPS', 'Series_Complete_Pop_Pct']]

df['Date'] = pd.to_datetime(df['Date'])
# df = df.sort_values(by=['FIPS', 'Date']).reset_index(drop=True)
df.fillna(0, inplace=True)

In [14]:
df = df[['Date', 'FIPS', 'Series_Complete_Pop_Pct']].pivot_table(
    values='Series_Complete_Pop_Pct',
    index=['FIPS'],
    columns='Date'
).reset_index().rename_axis(None, axis=1)
df.head()

,FIPS,2021-06-13 00:00:00,2021-06-14 00:00:00,2021-06-15 00:00:00,2021-06-16 00:00:00,2021-06-17 00:00:00,2021-06-18 00:00:00,2021-06-19 00:00:00,2021-06-20 00:00:00,2021-06-21 00:00:00,2021-06-22 00:00:00,2021-06-23 00:00:00,2021-06-24 00:00:00,2021-06-25 00:00:00,2021-06-26 00:00:00,2021-06-27 00:00:00,2021-06-28 00:00:00,2021-06-29 00:00:00,2021-06-30 00:00:00,2021-07-01 00:00:00,2021-07-02 00:00:00,2021-07-03 00:00:00,2021-07-04 00:00:00,2021-07-05 00:00:00,2021-07-06 00:00:00,2021-07-07 00:00:00,2021-07-08 00:00:00,2021-07-09 00:00:00,2021-07-10 00:00:00,2021-07-11 00:00:00,2021-07-12 00:00:00,2021-07-13 00:00:00,2021-07-14 00:00:00,2021-07-15 00:00:00,2021-07-16 00:00:00,2021-07-17 00:00:00,2021-07-18 00:00:00,2021-07-19 00:00:00,2021-07-20 00:00:00,2021-07-21 00:00:00,2021-07-22 00:00:00,2021-07-23 00:00:00,2021-07-24 00:00:00,2021-07-25 00:00:00,2021-07-26 00:00:00,2021-07-27 00:00:00,2021-07-28 00:00:00,2021-07-29 00:00:00,2021-07-30 00:00:00,2021-07-31 00:00:00,2021-08-01 00:00:00,2021-08-02 00:00:00,2021-08-03 00:00:00,2021-08-04 00:00:00,2021-08-05 00:00:00,2021-08-06 00:00:00,2021-08-07 00:00:00,2021-08-08 00:00:00,2021-08-09 00:00:00,2021-08-10 00:00:00,2021-08-11 00:00:00,2021-08-12 00:00:00,2021-08-13 00:00:00,2021-08-14 00:00:00,2021-08-15 00:00:00,2021-08-16 00:00:00,2021-08-17 00:00:00,2021-08-18 00:00:00,2021-08-19 00:00:00,2021-08-20 00:00:00,2021-08-21 00:00:00,2021-08-22 00:00:00,2021-08-23 00:00:00,2021-08-24 00:00:00,2021-08-25 00:00:00,2021-08-26 00:00:00,2021-08-27 00:00:00,2021-08-28 00:00:00,2021-08-29 00:00:00,2021-08-30 00:00:00,2021-08-31 00:00:00,2021-09-01 00:00:00,2021-09-02 00:00:00,2021-09-03 00:00:00,2021-09-04 00:00:00,2021-09-05 00:00:00,2021-09-06 00:00:00,2021-09-07 00:00:00,2021-09-08 00:00:00,2021-09-09 00:00:00,2021-09-10 00:00:00,2021-09-11 00:00:00,2021-09-12 00:00:00,2021-09-13 00:00:00,2021-09-14 00:00:00,2021-09-15 00:00:00,2021-09-16 00:00:00,2021-09-17 00:00:00,2021-09-18 00:00:00,2021-09-19 00:00:00,2021-09-20 00:00:00,2021-09-21 00:00:00,2021-09-22 00:00:00,2021-09-23 00:00:00,2021-09-24 00:00:00,2021-09-25 00:00:00,2021-09-26 00:00:00,2021-09-27 00:00:00,2021-09-28 00:00:00,2021-09-29 00:00:00,2021-09-30 00:00:00,2021-10-01 00:00:00,2021-10-02 00:00:00,2021-10-03 00:00:00,2021-10-04 00:00:00,2021-10-05 00:00:00,2021-10-06 00:00:00,2021-10-07 00:00:00,2021-10-08 00:00:00,2021-10-09 00:00:00,2021-10-10 00:00:00,2021-10-11 00:00:00,2021-10-12 00:00:00,2021-10-13 00:00:00,2021-10-14 00:00:00,2021-10-15 00:00:00,2021-10-16 00:00:00,2021-10-17 00:00:00,2021-10-18 00:00:00,2021-10-19 00:00:00,2021-10-20 00:00:00,2021-10-21 00:00:00,2021-10-22 00:00:00,2021-10-23 00:00:00,2021-10-24 00:00:00,2021-10-25 00:00:00,2021-10-26 00:00:00,2021-10-27 00:00:00,2021-10-28 00:00:00,2021-10-29 00:00:00,2021-10-30 00:00:00,2021-10-31 00:00:00,2021-11-01 00:00:00,2021-11-02 00:00:00,2021-11-03 00:00:00,2021-11-04 00:00:00,2021-11-05 00:00:00,2021-11-06 00:00:00,2021-11-07 00:00:00,2021-11-08 00:00:00,2021-11-09 00:00:00,2021-11-10 00:00:00,2021-11-11 00:00:00,2021-11-12 00:00:00,2021-11-13 00:00:00,2021-11-14 00:00:00,2021-11-15 00:00:00,2021-11-16 00:00:00,2021-11-17 00:00:00,2021-11-18 00:00:00,2021-11-19 00:00:00,2021-11-20 00:00:00,2021-11-21 00:00:00,2021-11-22 00:00:00,2021-11-23 00:00:00,2021-11-24 00:00:00,2021-11-25 00:00:00,2021-11-26 00:00:00,2021-11-27 00:00:00,2021-11-28 00:00:00,2021-11-29 00:00:00,2021-11-30 00:00:00,2021-12-01 00:00:00,2021-12-02 00:00:00,2021-12-03 00:00:00,2021-12-04 00:00:00,2021-12-05 00:00:00,2021-12-06 00:00:00,2021-12-07 00:00:00,2021-12-08 00:00:00,2021-12-09 00:00:00,2021-12-10 00:00:00,2021-12-11 00:00:00,2021-12-12 00:00:00,2021-12-13 00:00:00,2021-12-14 00:00:00,2021-12-15 00:00:00,2021-12-16 00:00:00,2021-12-17 00:00:00,2021-12-18 00:00:00,2021-12-19 00:00:00,2021-12-20 00:00:00,2021-12-21 00:00:00,2021-12-22 00:00:00,2021-12-23 00:00:00,2021-12-24 00:00:00,2021-12-25 00:00:00,2021-12-26 00:00:00,2021-12-27 00:00:00,2021-12-28 00:00:00,2021-12-29 00:

In [15]:
df.shape

(3224, 399)

In [16]:
def fill_missing_dates(data):
    df = data.copy()
    df = df.fillna(method='ffill').fillna(0)
    # this assumes the dates are along the column axis
    date_columns = sorted([pd.to_datetime(col) for col in df.columns if valid_date(col)])
    non_date_columns = [col for col in df.columns if col not in date_columns]
    df.columns = non_date_columns + date_columns

    total_days = (date_columns[1] - date_columns[0]).days
    missing_dates = []

    previous_date = date_columns[0]
    for date in date_columns[1:]:
        steps = (date - previous_date).days
        if steps > 1:
            for step in range(1, steps):
                missing_dates.append(previous_date+pd.to_timedelta(step, unit='D'))
        previous_date = date

    print(f'Number of missing dates {len(missing_dates)}')

    df = df.reindex(columns = ['FIPS'] + sorted(date_columns + missing_dates))
    df = df.T.fillna(method='ffill').T
    return df

In [17]:
# this was added because after May 2022, 
# vaccination data from CDC started having random missing dates
df = fill_missing_dates(df)
df.head(2)

Number of missing dates 173


,FIPS,2021-06-13 00:00:00,2021-06-14 00:00:00,2021-06-15 00:00:00,2021-06-16 00:00:00,2021-06-17 00:00:00,2021-06-18 00:00:00,2021-06-19 00:00:00,2021-06-20 00:00:00,2021-06-21 00:00:00,2021-06-22 00:00:00,2021-06-23 00:00:00,2021-06-24 00:00:00,2021-06-25 00:00:00,2021-06-26 00:00:00,2021-06-27 00:00:00,2021-06-28 00:00:00,2021-06-29 00:00:00,2021-06-30 00:00:00,2021-07-01 00:00:00,2021-07-02 00:00:00,2021-07-03 00:00:00,2021-07-04 00:00:00,2021-07-05 00:00:00,2021-07-06 00:00:00,2021-07-07 00:00:00,2021-07-08 00:00:00,2021-07-09 00:00:00,2021-07-10 00:00:00,2021-07-11 00:00:00,2021-07-12 00:00:00,2021-07-13 00:00:00,2021-07-14 00:00:00,2021-07-15 00:00:00,2021-07-16 00:00:00,2021-07-17 00:00:00,2021-07-18 00:00:00,2021-07-19 00:00:00,2021-07-20 00:00:00,2021-07-21 00:00:00,2021-07-22 00:00:00,2021-07-23 00:00:00,2021-07-24 00:00:00,2021-07-25 00:00:00,2021-07-26 00:00:00,2021-07-27 00:00:00,2021-07-28 00:00:00,2021-07-29 00:00:00,2021-07-30 00:00:00,2021-07-31 00:00:00,2021-08-01 00:00:00,2021-08-02 00:00:00,2021-08-03 00:00:00,2021-08-04 00:00:00,2021-08-05 00:00:00,2021-08-06 00:00:00,2021-08-07 00:00:00,2021-08-08 00:00:00,2021-08-09 00:00:00,2021-08-10 00:00:00,2021-08-11 00:00:00,2021-08-12 00:00:00,2021-08-13 00:00:00,2021-08-14 00:00:00,2021-08-15 00:00:00,2021-08-16 00:00:00,2021-08-17 00:00:00,2021-08-18 00:00:00,2021-08-19 00:00:00,2021-08-20 00:00:00,2021-08-21 00:00:00,2021-08-22 00:00:00,2021-08-23 00:00:00,2021-08-24 00:00:00,2021-08-25 00:00:00,2021-08-26 00:00:00,2021-08-27 00:00:00,2021-08-28 00:00:00,2021-08-29 00:00:00,2021-08-30 00:00:00,2021-08-31 00:00:00,2021-09-01 00:00:00,2021-09-02 00:00:00,2021-09-03 00:00:00,2021-09-04 00:00:00,2021-09-05 00:00:00,2021-09-06 00:00:00,2021-09-07 00:00:00,2021-09-08 00:00:00,2021-09-09 00:00:00,2021-09-10 00:00:00,2021-09-11 00:00:00,2021-09-12 00:00:00,2021-09-13 00:00:00,2021-09-14 00:00:00,2021-09-15 00:00:00,2021-09-16 00:00:00,2021-09-17 00:00:00,2021-09-18 00:00:00,2021-09-19 00:00:00,2021-09-20 00:00:00,2021-09-21 00:00:00,2021-09-22 00:00:00,2021-09-23 00:00:00,2021-09-24 00:00:00,2021-09-25 00:00:00,2021-09-26 00:00:00,2021-09-27 00:00:00,2021-09-28 00:00:00,2021-09-29 00:00:00,2021-09-30 00:00:00,2021-10-01 00:00:00,2021-10-02 00:00:00,2021-10-03 00:00:00,2021-10-04 00:00:00,2021-10-05 00:00:00,2021-10-06 00:00:00,2021-10-07 00:00:00,2021-10-08 00:00:00,2021-10-09 00:00:00,2021-10-10 00:00:00,2021-10-11 00:00:00,2021-10-12 00:00:00,2021-10-13 00:00:00,2021-10-14 00:00:00,2021-10-15 00:00:00,2021-10-16 00:00:00,2021-10-17 00:00:00,2021-10-18 00:00:00,2021-10-19 00:00:00,2021-10-20 00:00:00,2021-10-21 00:00:00,2021-10-22 00:00:00,2021-10-23 00:00:00,2021-10-24 00:00:00,2021-10-25 00:00:00,2021-10-26 00:00:00,2021-10-27 00:00:00,2021-10-28 00:00:00,2021-10-29 00:00:00,2021-10-30 00:00:00,2021-10-31 00:00:00,2021-11-01 00:00:00,2021-11-02 00:00:00,2021-11-03 00:00:00,2021-11-04 00:00:00,2021-11-05 00:00:00,2021-11-06 00:00:00,2021-11-07 00:00:00,2021-11-08 00:00:00,2021-11-09 00:00:00,2021-11-10 00:00:00,2021-11-11 00:00:00,2021-11-12 00:00:00,2021-11-13 00:00:00,2021-11-14 00:00:00,2021-11-15 00:00:00,2021-11-16 00:00:00,2021-11-17 00:00:00,2021-11-18 00:00:00,2021-11-19 00:00:00,2021-11-20 00:00:00,2021-11-21 00:00:00,2021-11-22 00:00:00,2021-11-23 00:00:00,2021-11-24 00:00:00,2021-11-25 00:00:00,2021-11-26 00:00:00,2021-11-27 00:00:00,2021-11-28 00:00:00,2021-11-29 00:00:00,2021-11-30 00:00:00,2021-12-01 00:00:00,2021-12-02 00:00:00,2021-12-03 00:00:00,2021-12-04 00:00:00,2021-12-05 00:00:00,2021-12-06 00:00:00,2021-12-07 00:00:00,2021-12-08 00:00:00,2021-12-09 00:00:00,2021-12-10 00:00:00,2021-12-11 00:00:00,2021-12-12 00:00:00,2021-12-13 00:00:00,2021-12-14 00:00:00,2021-12-15 00:00:00,2021-12-16 00:00:00,2021-12-17 00:00:00,2021-12-18 00:00:00,2021-12-19 00:00:00,2021-12-20 00:00:00,2021-12-21 00:00:00,2021-12-22 00:00:00,2021-12-23 00:00:00,2021-12-24 00:00:00,2021-12-25 00:00:00,2021-12-26 00:00:00,2021-12-27 00:00:00,2021-12-28 00:00:00,2021-12-29 00:

In [20]:
date_columns = [col for col in df.columns if valid_date(col)]
drop_columns = [
    col for col in date_columns
        if col < pd.to_datetime(start) or col > pd.to_datetime(end)
]
df.drop(columns=drop_columns, inplace=True)

In [22]:
df['FIPS'] = df['FIPS'].astype(int)
df.to_csv(f'{output_folder}/Vaccination.csv', index=False)